In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [33]:
def get_sigma_points(mu, sigma, lambd, alpha, beta):
    n = mu.shape[0]
    sigma_points = np.zeros((n, 2*n+1))
    
    sigma_points[:, [0]] = mu
    
    mroot = np.linalg.cholesky((n+lambd)*sigma)
    
    # compute sigma points
    for i in range(1, n+1):
        # nice indexing
        sigma_points[:,[i]] = mu + mroot[:, [i-1]]
    
    for i in range(n+1, 2*n+1):
        print(i, i-n+1)
        sigma_points[:,[i]] = mu - mroot[:, [i-n-1]]
        
    # compute weights for mean and covariance recovery
    w_m = np.zeros(2*n+1)
    w_c = np.zeros(2*n+1)
    
    w_m[0] = lambd / (n + lambd)
    w_c[0] = w_m[0] + (1 - alpha**2 + beta)
    weight = 1 / ( 2 * (n + lambd))
    for t in range(1,2*n+1):
        w_m[t] = weight
        w_c[t] = weight
        
    return sigma_points, w_m, w_c

In [4]:
def recover_gaussian(sigma_points, w_m, w_c):
    n = sigma_points.shape[0]
    n_col = sigma_points.shape[1]
    
    mu = np.zeros((n, 1))
    
    # Recover mean
    for i in range(0, n_col):
        mu = mu + w_m[i] * sigma_points[:, [i]]
        
    # Recover covariance
    sigma = np.zeros((n, n))
    for i in range(0, n_col):
        sigma = sigma + w_c[i]*(sigma_points[:, [i]] - mu) * (sigma_points[:, [i]] - mu).T

    return mu, sigma

In [5]:
def transform_linear(points):
    points[0:1, :] = points[0:1, :] + 1
    points[1:2, :] = points[1:2, :] + 2
    
    return points

In [6]:
def transform_motion_model(points, command):
    return points

In [36]:
sigma = np.eye(3) * 0.1
mu = np.matrix([0, 0, 3]).T

n = mu.shape[0]
alpha = 0.9
beta = 2
kappa = 1
lambd = alpha*alpha*(n+kappa)-n

print(mu)
print(sigma)
print(20*'=')

sigma_points, w_m, w_c = get_sigma_points(mu, sigma, lambd, alpha, beta)
print(sigma_points)
print(w_m)
print(w_c)

transform_linear(sigma_points)

mu, sigma = recover_gaussian(sigma_points, w_m, w_c)
print(20*'=')
print(mu)
print(sigma)

[[0]
 [0]
 [3]
 [4]
 [5]
 [6]]
[[ 0.1  0.   0.   0.   0.   0. ]
 [ 0.   0.1  0.   0.   0.   0. ]
 [ 0.   0.   0.1  0.   0.   0. ]
 [ 0.   0.   0.   0.1  0.   0. ]
 [ 0.   0.   0.   0.   0.1  0. ]
 [ 0.   0.   0.   0.   0.   0.1]]
[[ 0.          0.75299402  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.75299402  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 3.          3.          3.          3.75299402  3.          3.          3.
   0.          0.          0.          0.          0.          0.        ]
 [ 4.          4.          4.          4.          4.75299402  4.          4.
   0.          0.          0.          0.          0.          0.        ]
 [ 5.          5.          5.          5.          5.          5.75299402
   5.          0.          0.          0.          0.          0.          0.       

[1, 2, 3]